In [15]:
import argparse
import numpy as np
import os
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import tensorflow_model_optimization as tfmot

In [2]:
model = "MLP"
#labels = 0
#labels = 1
labels = 2
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

In [5]:
csv_path = 'I:\Polito\ML4IOT\CODES TO DO\Lab3\ex1\jena_climate_2009_2016.csv'
df = pd.read_csv(csv_path)

In [6]:
column_indices = [2, 5]
#column_indices = 5
columns = df.columns[column_indices]
data = df[columns].values.astype(np.float32)

In [7]:
n = len(data)
train_data = data[0:int(n*0.7)]
val_data = data[int(n*0.7):int(n*0.9)]
test_data = data[int(n*0.9):]

In [8]:
mean = train_data.mean(axis=0)
std = train_data.std(axis=0)

In [9]:
input_width = 6
LABEL_OPTIONS = labels

In [10]:
class WindowGenerator:
    def __init__(self, input_width, label_options, mean, std):
        self.input_width = input_width
        self.label_options = label_options
        self.mean = tf.reshape(tf.convert_to_tensor(mean), [1, 1, 2])
        self.std = tf.reshape(tf.convert_to_tensor(std), [1, 1, 2])

    def split_window(self, features):
        inputs = features[:, :-1, :]

        if self.label_options < 2:
            labels = features[:, -1, self.label_options]
            labels = tf.expand_dims(labels, -1)
            num_labels = 1
        else:
            labels = features[:, -1, :]
            num_labels = 2

        inputs.set_shape([None, self.input_width, 2])
        labels.set_shape([None, num_labels])

        return inputs, labels

    def normalize(self, features):
        features = (features - self.mean) / (self.std + 1.e-6)

        return features

    def preprocess(self, features):
        inputs, labels = self.split_window(features)
        inputs = self.normalize(inputs)
        return inputs, labels

    def make_dataset(self, data, train):
        ds = tf.keras.preprocessing.timeseries_dataset_from_array(
                data=data,
                targets=None,
                sequence_length=input_width+1,
                sequence_stride=1,
                batch_size=32)
        ds = ds.map(self.preprocess)
        ds = ds.cache()
        if train is True:
            ds = ds.shuffle(100, reshuffle_each_iteration=True)

        return ds

In [11]:
class MultipleOutputforMAE(tf.keras.metrics.Metric):
  def __init__(self, name='MAE', **kwargs):
      super().__init__(name=name, **kwargs)
      self.total = self.add_weight(name='total', initializer='zeros',shape=(2,))
      self.count = self.add_weight('count',initializer = 'zeros')
  def update_state(self, y_true, y_pred, sample_weight=None):
    error = tf.abs(y_pred - y_true)
    error = tf.reduce_mean(error,axis = 0)
    self.total.assign_add(error)
    self.count.assign_add(1.)

  def reset_states(self):
    self.count.assign(tf.zeros_like(self.count))
    self.total.assign(tf.zeros_like(self.total))
    return

  def result(self):
    result = tf.math.divide_no_nan(self.total,self.count)
    return result

In [12]:
generator = WindowGenerator(input_width, LABEL_OPTIONS, mean, std)
train_ds = generator.make_dataset(train_data, True)
val_ds = generator.make_dataset(val_data, False)
test_ds = generator.make_dataset(test_data, False)

In [33]:
MLPmodel = keras.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(2)
])

In [34]:
pruning_params = {'pruning_schedule':
    tfmot.sparsity.keras.PolynomialDecay(
    initial_sparsity=0.30,
    final_sparsity=0.8,
    begin_step=len(train_ds)*5,
    end_step=len(train_ds)*15)
    }

In [36]:
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude
model = prune_low_magnitude(MLPmodel, **pruning_params)

In [37]:
callbacks = [tfmot.sparsity.keras.UpdatePruningStep()]

In [42]:
 if labels != 2:
    metrics = [tf.metrics.MeanAbsoluteError()]
else:
    metrics = [MultipleOutputforMAE()]


In [46]:
model.compile(optimizer='adam',
                  loss=tf.keras.losses.MAE,
                  metrics=metrics)


In [47]:
input_shape = [32, 6, 2]
model.build(input_shape)
model.fit(train_ds, epochs=1, validation_data=val_ds, callbacks=callbacks)

9186/9200 [============================>.] - ETA: 0s - loss: 1.9277 - MAE: 1.9276

C:\Users\imanp\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py:1841: UserWarning: Metric MultipleOutputforMAE implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  m.reset_state()


9200/9200 [==============================] - 21s 2ms/step - loss: 1.9258 - MAE: 1.9257 - val_loss: 0.6290 - val_MAE: 0.6290


In [48]:
model = tfmot.sparsity.keras.strip_pruning(model)

In [50]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 12)                0         
                                                                 
 dense_3 (Dense)             (None, 128)               1664      
                                                                 
 dense_4 (Dense)             (None, 128)               16512     
                                                                 
 dense_5 (Dense)             (None, 2)                 258       
                                                                 
Total params: 18,434
Trainable params: 18,434
Non-trainable params: 0
_________________________________________________________________


In [55]:
saved_model_dir = 'magnitude-based model'
run_model = tf.function(lambda x: model(x))
concrete_func = run_model.get_concrete_function(tf.TensorSpec([1, 6, 2],
tf.float32))
model.save(saved_model_dir, signatures=concrete_func)

INFO:tensorflow:Assets written to: magnitude-based model\assets


In [58]:
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()
tflite_model_dir = 'I:\Polito\ML4IOT\CODES TO DO\Lab4\ex3\magnitude-based model.tflite'
with open(tflite_model_dir, 'wb') as fp:
    fp.write(tflite_model)

In [59]:
import zlib
tflite_model = converter.convert()
with open(tflite_model_dir, 'wb') as fp:
    tflite_compressed = zlib.compress(tflite_model)
    fp.write(tflite_compressed)